In [ ]:
# Must set these constants before running this helper.
# TEST_NAME
# OUT_FILE


WINDOW_SIZE = '1D'

In [ ]:
# All imports live here.

import pandas as pd
pd.options.plotting.backend = 'plotly'

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Load and parse raw data.
with open('../../data/raw/playwright.json') as f:
    tests = pd.read_json(f)
print('Loaded tests with %d rows. Columns:' % (tests.shape[0],))
print(tests.dtypes)

In [ ]:
# Augmented columns
tests['workflow_created_date'] = tests['workflow_created_at'].dt.normalize()
tests['test_uid'] = tests['composite'].astype(str) + '/' + tests['pw_suite_title'].astype(str) + '/' + tests['pw_spec_title'].astype(str)

print(tests.dtypes)

In [ ]:
all_tests = tests
# Filter to relevant tests
cond = ((tests['workflow_event'] == 'push') & (tests['workflow_head_branch'] == 'main'))
tests = tests[cond]

In [ ]:
df = tests
df = df[df['test_uid'] == TEST_NAME]
df['url'] = df.apply(lambda x: 'https://github.com/Azure/communication-ui-library/actions/runs/{}'.format(x['workflow_id'],), axis=1)
df = df.sort_values(by='workflow_created_at', ascending=False)
df = df[df['pw_result_status'] == 'failed']
df = df[['workflow_created_date', 'workflow_id', 'url', 'workflow_id', 'build_flavor', 'composite', 'pw_test_project_name', 'pw_result_retry']]
with open('./{}.csv'.format(OUT_FILE), 'w') as f:
    df.to_csv(f)

In [ ]:
def summed_counts(df):
    df = df.groupby(['workflow_created_date', 'pw_test_project_name']).count()
    df = df['workflow_id'].unstack()
    df = df.rolling(WINDOW_SIZE).sum()
    return df
    
def pass_perc(df):
    df = df[df['test_uid'] == TEST_NAME]
    df_total = summed_counts(df)
    df_failed = summed_counts(df[df['pw_result_status'] == 'failed'])

    df_total = df_total.stack()
    df_failed = df_failed.stack()
    df_pass_perc = df_total.sub(df_failed, fill_value=0) / df_total
    df_pass_perc.fillna(1)
    df_pass_perc = df_pass_perc[df_pass_perc.notna()]*100
    df_pass_perc = df_pass_perc.unstack()
    return df_pass_perc



In [ ]:
df_pass_perc = pass_perc(tests[tests['pw_result_retry'] == 2])

df = df_pass_perc
fig = df.plot(title='Success rate of test despite retries (postsubmit)', labels=dict(workflow_created_date='workflow created within last 1 days of', value='Success ratio (percent)', pw_test_project_name='Browser'))
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
df_pass_perc = pass_perc(tests)

df = df_pass_perc
fig = df.plot(title='Success rate of test (postsubmit)', labels=dict(workflow_created_date='workflow created within last 1 days of', value='Success ratio (percent)', pw_test_project_name='Browser'))
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
df_pass_perc = pass_perc(all_tests)

# Failure of presumit tests can be for non-flakiness reasons (i.e., bad PR rejected by CI Yay!)
# This is added just for comparison.
df = df_pass_perc
fig = df.plot(title='Success rate of test (including presubmit)', labels=dict(workflow_created_date='workflow created within last 1 days of', value='Success ratio (percent)', pw_test_project_name='Browser'))
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
for t in tests['test_uid'].unique():
    print(t)